In [1]:
import gym

import numpy as np
import random
import time

from IPython.display import clear_output

In [2]:
# Environment
env = gym.make("FrozenLake-v0")

# Q-table
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

In [3]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [4]:
rewards_all_episodes = []

for episode in range(num_episodes):
    state = env.reset()
    done = False
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 

        # Pick action with exploration-exploitation trade-off
        exploration_rate_threshold = random.uniform(0, 1)
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:]) 
        else:
            action = env.action_space.sample()
            
        # Act the picked action
        new_state, reward, done, info = env.step(action)
        
        # Update Q-table for Q(s,a)
#         q_table[state, action] = q_table[state, action] * (1 - learning_rate) + learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        loss = (reward + discount_rate * np.max(q_table[new_state, :])) - q_table[state, action]
        q_table[state, action] = q_table[state, action] + learning_rate * loss
        
        # Transit to next state
        state = new_state
        rewards_current_episode += reward
        
        if done == True: 
            break
            
    # Exploration rate decay after each episode
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    rewards_all_episodes.append(rewards_current_episode)

In [5]:
# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes),num_episodes/1000)
count = 1000

print("********Average reward per thousand episodes********\n")
for r in rewards_per_thousand_episodes:
    print(count, ": ", str(sum(r/1000)))
    count += 1000

********Average reward per thousand episodes********

1000 :  0.047000000000000035
2000 :  0.20300000000000015
3000 :  0.43900000000000033
4000 :  0.5760000000000004
5000 :  0.6330000000000005
6000 :  0.6350000000000005
7000 :  0.6910000000000005
8000 :  0.6730000000000005
9000 :  0.6530000000000005
10000 :  0.6980000000000005


In [6]:
# Print updated Q-table
print("\n\n********Q-table********\n")
print(q_table)



********Q-table********

[[0.55530395 0.52066673 0.54462806 0.51439336]
 [0.31547959 0.24655155 0.22465714 0.47420465]
 [0.41637631 0.27404382 0.29160363 0.27317921]
 [0.10762822 0.27031745 0.04834756 0.03520697]
 [0.57220074 0.34052224 0.45389697 0.36875956]
 [0.         0.         0.         0.        ]
 [0.41691315 0.08746783 0.26255645 0.13449501]
 [0.         0.         0.         0.        ]
 [0.44042675 0.36747831 0.45935055 0.59994361]
 [0.2288726  0.66450999 0.34264537 0.53377356]
 [0.58856942 0.48959266 0.47024751 0.27457059]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.53878459 0.54809001 0.75186742 0.44196648]
 [0.71184131 0.84904122 0.81534524 0.74631616]
 [0.         0.         0.         0.        ]]


In the Q-table, rows with only zeros are either H or G states. Because, once the agent reaches there, the episode will end and so, Q-values at those states will never be updated. 

I speculate that the final row is the G state. Its previous state has very high Q-values, which suggests that this state is close to the G state on the map.

# Play with the learned Q-table

In [7]:
env.reset()

for episode in range(5):
    state = env.reset()
    step = 0
    done = False
    print("****************************************************")
    print("EPISODE ", episode)

    for step in range(max_steps_per_episode):
        
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(q_table[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        if done:
            # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
            env.render()
            
            # We print the number of step it took.
            print("Number of steps", step)
            break
        state = new_state
env.close()

****************************************************
EPISODE  0
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 43
****************************************************
EPISODE  1
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 8
****************************************************
EPISODE  2
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 26
****************************************************
EPISODE  3
  (Left)
SFFF
FHFH
FFFH
HFFG
Number of steps 12
****************************************************
EPISODE  4
  (Down)
SFFF
FHFH
FFFH
HFFG
Number of steps 38


In [8]:
for episode in range(3):
    state = env.reset()
    done = False
    print("*****EPISODE ", episode+1, "*****\n\n\n\n")
    time.sleep(1)
    
    for step in range(max_steps_per_episode):        
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        
        action = np.argmax(q_table[state,:])        
        new_state, reward, done, info = env.step(action)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("****You reached the goal!****")
                time.sleep(3)
            else:
                print("****You fell through a hole!****")
                time.sleep(3)
                clear_output(wait=True)
            break
        
        state = new_state
        
env.close()

  (Down)
SFFF
FHFH
FFFH
HFFG
****You reached the goal!****
